### Clean the Annotations
- annotations.csv lack with GENDER and other attributes
- annotations.csv includes GENDER but lost other attributes

In [11]:
import os
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import scipy.io
from scipy.io import loadmat, savemat
from maskable_list import MaskableList


ann_df = pd.read_csv('/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/annotations.csv')
ann_new_df = pd.read_csv('/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/annontations_New.csv')

In [4]:
print(ann_df.shape)
print(ann_new_df.shape)
# pd.concat([ann_df, ann_new_df], axis=1)
ann_combined = pd.merge(ann_df, ann_new_df, on='Subject', how='inner')
print(ann_combined.shape)

(1206, 24)
(1206, 573)
(1206, 596)


### Based on the center coordinate matrix (i.e. XXROI_center.mat) fitler the subjects

In [5]:
ROI_center_132 = scipy.io.loadmat('/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/132ROI_center.mat')
# ROI_center_82 = scipy.io.loadmat('/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/82ROI_center.mat')

subject_ids = [] 
# print(ROI_center_132['ROI_center'][0][1])

for subject in ROI_center_132['ROI_center']:
    if subject[1].shape[0] > 0: # subject[1] is ndarray(132, 3)
        subject_ids.append(int(subject[0][0])) # The subject id, type = ndarray -> int
        
print("The number of subjects should be", len(subject_ids))
ann_filtered = ann_combined[ann_combined['Subject'].isin(subject_ids)]
print(ann_filtered.shape)
ann_filtered.to_csv('/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/filtered_annotations.csv', index=False)

The number of subjects should be 1040
(1040, 596)


In [6]:
ann_filtered['Gender'].unique()
print(len(ann_filtered['Gender']))
print(len(ann_filtered['Subject']))

1040
1040


### Transform the brain data
- functional network: derivedresting-state fMRI (1063 subjects)
    - Normalized weighted adjacency matrix for each subject (.mat)
    - Filename is the subject ID
- structural network: derived from diffusion MRI (1063 subjects)
    - Adjacency matrix for each subject

In [20]:
# directory = '/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/functional_network_132_0'
# replace_nan_in_mat(directory)
# adj_dict = loadmat(f'{directory}') 

In [7]:
def load_adj_label_HCP(data_dir, label_dir, modality='dti'):
    """
    load adjacency matrix from raw matlab (G, N, N) & load the label from csv file (G,).
    We need to make them a correct pair!

    Args:
        data_dir: root dir to åsave the dataset
        dataset: name of the dataset
        modality: 'dti' by default (structural MRI, SMRI) or 'func' (FMRI)
        is_dup: Whether the label file has duplicated subject

    returns:
        adj: (G, N, N), G is the graph for cleaned subjects
        y: (G,)
    """

    # Load the dict of adj matrix (including all the subjects such dirty ones, duplicated ones)

    # dirty currently
    # Don't no if need to transpose the adj matrix
    # adj = adj_dict[adj_key].transpose((2, 1, 0))  # np.narray (# dirty graphs/substances, # node, # node)

    # Find the Label from the csv file
    #label_file_name = os.path.splitext(label_dir)[0]
    # demo_file = f'{label_file_name}' + ('_dup.csv' if Is_dup else '.csv')
    
    demo_df = pd.read_csv(label_dir)
    select_cols = ['Subject', 'Race', 'Gender', 'Age']
    demo_df = demo_df[select_cols]
    
    adj_list = []
    label_list = []
    # Get the corresponding adj matrix
    for idx, sub in enumerate(demo_df['Subject']):
        if not os.path.isfile(f'{data_dir}/{sub}.mat'):
            print(f'{sub}.mat does not exist in the adj path!')
            continue

        ##dirty currently
        ## adj = adj_dict[adj_key].transpose((2, 1, 0))  # np.narray (# dirty graphs/substances, # node, # node)
        ## adj = extract_knn(a1, args.top_k) # when args.top_k == 0, return the original a1

        ## Get the corresponding adj from adj path
        ## adj = adj[sub_list, :, :]
        adj_dict = loadmat(f'{data_dir}/{sub}')
        adj_key = list(adj_dict.keys())[-1]
        adj = torch.Tensor(adj_dict[adj_key])

        # Make the negative weights positive for fmri
        if modality == 'func':
            adj[adj < 0] = -adj[adj < 0]
            mask = torch.isnan(adj)
            adj[mask] = 0
        elif modality == 'dti':  # Min-Max normalization for smri
            adj = (adj - adj.min()) / (adj.max() - adj.min())
        adj_list.append(adj)

        # Process labels
        gender = demo_df.loc[idx, 'Gender']
        label = 0 if gender == 'M' else 1
        label_list.append(label)
        
    adj_tensor = torch.stack(adj_list) if adj_list else torch.empty(0)
    y = torch.tensor(label_list, dtype=torch.long) if label_list else torch.empty(0, dtype=torch.long)
    # y = torch.tensor(label_df).long().flatten()  # labels for all the graph (#graph)

    return adj_tensor, y

150019.mat does not exist in the adj path!
173233.mat does not exist in the adj path!
248238.mat does not exist in the adj path!
693461.mat does not exist in the adj path!
995174.mat does not exist in the adj path!
adj matrix size: torch.Size([1035, 132, 132])
label size: torch.Size([1035])


In [15]:
def generate_node_features(adj, node_feature_type='identity', f_dim=None):
    """
    Args:
        adj: weighted adjacency matrix (G, N, N), G is the graph for cleaned subjects
        node_feature_type: The way to generate the node features
        f_dim: optional, not all of type will use it
    returns:
        x1: node feature (#graph, #node, #f_dim)
    """
    n_graph = adj.shape[0]
    n_node = adj.shape[1]
    if f_dim is None:
        f_dim = n_node

    # construct node features X
    if node_feature_type == 'identity':
        x1 = torch.ones(n_graph, n_node, f_dim)
    # elif node_feature_type == 'onehot':   #  one-hot
    #     # (#graph, #node, #f_dim), where the feature is one hot vector, #f_dim=#nodes
    #     # for each graph, it is a dialog matrix, so for each row (node), it is one-hot vector
    #     x = torch.cat([torch.diag(torch.ones(n_node))] * n_graph).reshape([n_graph, n_node, -1])
    #     x1 = x.clone()  # (#graph, #node, #f_dim=#nodes)

    # elif node_feature_type == 'degree':
    #     # use degree to represent its feature
    #     a1b = (adj != 0).float()  # 0./1. tensor with (#graph, #node, #node)
    #     x1 = a1b.sum(dim=2, keepdim=True)  # (#graph, #node, #f_dim=1)
    #################################################################################
    elif node_feature_type == 'adj':  # use the adj matrix directly
        x1 = adj.float()  # (#graph, #node, #nodes)
    #################################################################################
    # elif node_feature_type == 'LDP':  # degree profile
    #     a1b = (adj != 0).float()  # 0./1. tensor with (#graph, # node, # node)
    #     x1 = []
    #     for i in range(n_graph):
    #         x1.append(LDP(nx.from_numpy_array(a1b[i].numpy())))

    # elif node_feature_type == 'eigen':
    #     _, x = LA.eig(adj.numpy())  # where is x1?
    #     x1 = x  # we add

    # elif node_feature_type == 'degree_bin':
    #     a1b = (adj != 0).float()
    #     x1 = binning(a1b.sum(dim=2))  # (#graph, # node, # 1)
    #################################################################################
    elif node_feature_type == 'gaussian':  # standard normalization (mu=0,std=1)
        x1 = torch.normal(mean=0., std=1., size=(n_graph, n_node, f_dim))
        # x1 = torch.randn(n_graph, n_node, f_dim) # equivalent
    else:
        raise Exception("The type to generate node features is not supported")
    #################################################################################
    # elif node_feature_type == 'node2vec':
    #     X = np.load(f'./{args.dataset_name}_{args.modality}.emb', allow_pickle=True).astype(np.float32)
    #     x1 = torch.from_numpy(X)

    x1 = torch.Tensor(x1).float()
    return x1


# mark the edge index in tiled n^2 vector
def generate_edge_flag(n_node, edge_index):
    """
    Args:
        edge_index: (2, E)

    returns:
        edge_flag: (n_node*n_node, ), 1-dim, tensor. if non-zero, edge_flag is True, else False, bool Tensor
    """
    edge_flag = np.full((n_node ** 2,), False)
    n_edge = edge_index.shape[1]
    for i in range(n_edge):
        source = edge_index[0, i]
        target = edge_index[1, i]
        new_index = source * n_node + target
        edge_flag[new_index] = True

    edge_flag = torch.from_numpy(edge_flag)
    return edge_flag


def dense_adj_2_COO(adj):
    """
    Args:
        adj: dense adjacency matrix, (G, N, N)

    returns:
        ls_edge_index is list of the COO connectivity, ls_edge_weight is list the weight of each connected edge (scalar),
        ls_edge_flag is the list of flattened binary adj matrix.
        All the elements in the list is tensor

        ls_edge_index: 0/1 entries, [(2, E), ...], where len is G
        ls_edge_weight: [(E), ...], where len is G
        ls_edge_flag: binary entries, [(N*N, ), ..], where len is G
    """
    ls_edge_index = []
    ls_edge_weight = []  # Get it from the adj matrix
    ls_edge_flag = []
    n_graph = adj.shape[0]
    n_node = adj.shape[1]
    for i in range(n_graph):  # For each graph
        edge_index, edge_weight = dense_to_sparse(adj[i])  # (2, E), (E)
        # print("I am here. {}".format(i))
        # input(binary, 0/1): (2, E) -> output(binary, 0/1): (N*N), still 1-dim
        edge_flag = generate_edge_flag(n_node, edge_index)  # For each graph, flattened binary adj matrix (N*N)/flattened edge_index

        ls_edge_index.append(edge_index)
        ls_edge_weight.append(edge_weight)
        ls_edge_flag.append(edge_flag)

    return ls_edge_index, ls_edge_weight, ls_edge_flag


def build_dataset(x, ls_edge_index, ls_edge_weight, ls_edge_flag, y, adj):
    """
    Args: (all of the elements are tensor)
        x: node feature tensor (G, N, F)
        ls_edge_index: 0/1 entries, [(2, E), ...], where len is G
        ls_edge_weight: [(E), ...], where len is G
        ls_edge_flag: 0/1 entries, [(N*N, ), ..], where len is G, bool Tensor
        y: (G,), labels of all the graphs
        adj: (G, N, N), G is the graph for cleaned subjects
    returns:

    """
    data_list = MaskableList([])
    n_graph = y.shape[0]
    for idx in range(n_graph):
        data = Data(x=x[idx], edge_index=ls_edge_index[idx],
                    edge_weight=ls_edge_weight[idx], edge_attr=ls_edge_weight[idx],
                    edge_flag=ls_edge_flag[idx], y=y[idx], adj=adj[idx])
        data_list.append(data)

    return data_list


In [20]:
import time
start = time.time()
# sequence for data loading
data_dir = '/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/functional_network_132_0'
label_dir = '/Users/lihanzhao/Desktop/W&M/S24/BrainGNN/HCP_Data/filtered_annotations.csv'
f_dim = None
node_feature_type = 'gaussian'
print("Get adj & y")
adj, y = load_adj_label_HCP(data_dir, label_dir, modality='func')
print('adj matrix size:', adj.shape)
print('label size:', y.shape)
print()

print("get node features")
x = generate_node_features(adj=adj, node_feature_type=node_feature_type, f_dim=f_dim)

print("get edge characters in COO format")
ls_edge_index, ls_edge_weight, ls_edge_flag = dense_adj_2_COO(adj)

print("getting data list for dataset")
data_list = build_dataset(x, ls_edge_index, ls_edge_weight, ls_edge_flag, y, adj)
print("Done.")

end = time.time()
print("The total time for data loading is {} minutes".format((end-start)/60))
print()
print(adj.shape)
print(y.shape)
print(len(data_list))

getting data list for dataset
Done.
The total time for data loading is 0.002062666416168213 minutes

torch.Size([1035, 132, 132])
torch.Size([1035])
1035
